In [1]:
import sbol2
import plateo
import plateo.containers
import plateo.tools
import warnings
import pandas as pd
import uuid
from typing import List, Dict
from sbolParserApi import ParserSBOL
import numpy as np
import copy
from rdflib import URIRef
import posixpath

In [11]:
def enumerator(
    doc: sbol2.document.Document,
    derivation: sbol2.combinatorialderivation.CombinatorialDerivation
) -> List[sbol2.componentdefinition.ComponentDefinition]:
    parents = []
    template = doc.getComponentDefinition(derivation.masterTemplate)
    templateCopy = copy.deepcopy(template)
    parents.append(templateCopy)
    for vc in derivation.variableComponents:
        newParents = []
        for parent in parents:
            for children in group(collectVariants(vc)):
                varDisplayId = concatenateChildrenDisplayId(children)
                # Create parent copy
                uniqueId = getUniqueDisplayId(None, None, parent.displayId + "_" + varDisplayId, parent.version, "CD", doc)
                newParent = createCopy(parent, uniqueId, "1")
                doc.add(newParent)
                # Set newParent to existing CD
                newParent = doc.getComponentDefinition(newParent.identity)
                # Add children
                addChildren(template, template.component[vc.variable], newParent, children)

    print("Parents", parents)
    return

def addChildren(
    originalTemplate: sbol2.componentdefinition.ComponentDefinition,
    originalComponent: sbol2.component.Component,
    newParent: sbol2.componentdefinition.ComponentDefinition,
    children: List[sbol2.componentdefinition.ComponentDefinition]
):
    newComponent = newParent.components[originalComponent.identity]
    newComponent.wasDerivedFrom = originalComponent.identity
    if children is None:
        removeConstraintReferences(newParent, newComponent)
        for sa in newParent.sequenceAnnotations:
            if sa.component is not None and sa.component == newComponent.identity:
                newParent.sequenceAnnotations.remove(sa.identity)
        newParent.components.remove(newComponent.identity)
        return
    first = True
    for child in children:
        if first:
            newComponent.definition = child.identity
            first = False
        else:
            uniqueId = getUniqueDisplayId(newParent, None, child.displayId + "_Component", "1", "Component", None)
            link = newParent.components.create(uniqueId)
            link.definition = child.persistentIdentity
            link.access = SBOL_ACCESS_PUBLIC
            link.version = child.version
            link.wasDerivedFrom = originalComponent.identity
        


def removeConstraintReferences(
    newParent: sbol2.componentdefinition.ComponentDefinition,
    newComponent: sbol2.component.Component
):
    subj = None
    obj = None
    for sc in newParent.sequenceConstraints:
        if sc.subject == newComponent.identity:
            obj = newParent.components[sc.object]
            if subj is not None:
                sc.subject = subj
                obj = None
                subj = None
            else:
                newParent.sequenceConstraints.remove(sc.identity)
        if sc.object == newComponent.identity:
            subj = newParent.components[sc.subject]
            if obj is not None:
                sc.object = obj
                obj = None
                subj = None
            else:
                newParent.sequenceConstraints.remove(sc.identity)

def createCopy(
    obj: sbol2.object.SBOLObject,
    displayId: str,
    version: str
) -> sbol2.object.SBOLObject:
    oldDisplayId = obj.displayId
    oldPersistentId = obj.persistentIdentity
    newPersistentId = URIRef(oldPersistentId.replace(oldDisplayId, displayId))
    newIdentity = URIRef(posixpath.join(newPersistentId, version))
    newDisplayId = URIRef(displayId)
    objCopy = copy.deepcopy(obj)
    objCopy.displayId = newDisplayId
    objCopy.version = version
    objCopy.persistentIdentity = newPersistentId
    objCopy.identity = newIdentity
    return objCopy

def getUniqueDisplayId(
    comp: sbol2.componentdefinition.ComponentDefinition = None,
    derivation: sbol2.combinatorialderivation.CombinatorialDerivation = None,
    displayId: str = None,
    version: str = None,
    dataType: str = None,
    doc: sbol2.document.Document = None
) -> str:
    i = 1
    if dataType == "CD":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        while doc.find(uniqueUri):
            i += 1
            uniqueUri = sbol2.getHomespace + "%s_%d/%s" %(displayId, i, version)
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "SequenceAnnotation":
        while displayId in [sa.displayId for sa in comp.sequenceAnnotations]:
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "SequenceConstraint":
        while displayId in [sc.displayId for sc in comp.sequenceConstraints]:
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "Component":
        while displayId in [c.displayId for c in comp.components]:
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "Sequence":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        while doc.find(uniqueUri):
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    # TODO: Range
    elif dataType == "CombinatorialDerivation":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        while doc.find(uniqueUri):
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "VariableComponent":
        while displayId in [vc.displayId for vc in derivation.variableComponents]:
            i += 1
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    else:
        raise ValueError("")

def concatenateChildrenDisplayId(
    children: List[sbol2.componentdefinition.ComponentDefinition]
) -> str:
    concDisplayId = ""
    for child in children:
        concDisplayId = concDisplayId + child.displayId
    return concDisplayId

def collectVariants(
    vc: sbol2.combinatorialderivation.VariableComponent
) -> List[sbol2.componentdefinition.ComponentDefinition]:
    variants = []
    # Add all variants
    variants.extend(vc.variants)
    # Add all variants from Variant Collections
    for c in vc.variantCollections:
        for tl in c.members:
            # FIXME: MAYBE?
            if type(tl) == sbol2.componentdefinition.ComponentDefinition:
                variants.add(sbol2.componentdefinition.ComponentDefinition(tl))
    for derivation in vc.variantDerivations:
        variants.extend(enumerator(doc, derivation))
    return variants

def group(
    variants: List[sbol2.componentdefinition.ComponentDefinition],
    repeat: str
) -> List[List[sbol2.componentdefinition.ComponentDefinition]]:
    groups = []
    for cd in variants:
        group = []
        group.add(cd)
        groups.add(group)
    if repeat == "http://sbols.org/v2#one":
        return groups
    if repeat == "http://sbols.org/v2#zeroOrOne":
        groups.add([])
        return groups
    groups.clear()
    generateCombinations(groups, variants, 0, [])
    if repeat == "http://sbols.org/v2#oneOrMore":
        return groups
    if repeat == "http://sbols.org/v2#zeroOrMore":
        groups.add([])
        return groups

def generateCombinations(
    groups: List[List[sbol2.componentdefinition.ComponentDefinition]],
    variants: List[sbol2.componentdefinition.ComponentDefinition],
    i: int,
    sets: List[sbol2.componentdefinition.ComponentDefinition]
):
    if i == len(variants):
        if not sets:
            groups.add(sets)
        return
    no = sets.copy()
    generateCombinations(groups, variants, i+1, no)
    yes = sets.copy()
    yes.add(variants[i])
    generateCombinations(groups, variants, i+1, yes)


In [44]:
filepath = "../examples/sbol/combinatorial_1var.xml"
doc = sbol2.Document(filepath)
sbol2.setHomespace('http://www.dummy.org/')
sbol2.Config.setOption('sbol_compliant_uris', True)
sbol2.Config.setOption('sbol_typed_uris', True)
parser = ParserSBOL(doc)
derivation = doc.combinatorialderivations.get("http://www.dummy.org/design_CombinatorialDerivation/1")
cd = doc.componentDefinitions.get("http://www.dummy.org/design/1")
# enumerator(derivation)
cd_copy = copy.deepcopy(cd)
oldPersistentId = cd_copy.persistentIdentity
newPersistentId = URIRef(oldPersistentId.replace(cd_copy.displayId, "help"))
newIdentity = URIRef(posixpath.join(newPersistentId,"1"))
oldDisplayId = cd_copy.displayId
newDisplayId = oldDisplayId.replace(oldDisplayId, "help")
cd_copy.identity = newIdentity
cd_copy.persistentIdentity = newPersistentId
cd_copy.displayId = newDisplayId
print(cd_copy.persistentIdentity)
print(cd_copy.identity)
print(cd_copy.displayId)
doc.add(cd_copy)
cd_get = doc.getComponentDefinition(cd_copy.identity)
for sc in cd.sequenceConstraints:
    if sc.identity == "http://www.dummy.org/design/design_SequenceConstraint/1":
        sc.subject = cd.components["http://www.dummy.org/design/Pro_Component/1"]
        print("me")
for sc in cd.sequenceConstraints:
    if sc.identity == "http://www.dummy.org/design/design_SequenceConstraint/1":
        cd.sequenceConstraints.remove(sc.identity)
        print("me")
# cd.sequenceConstraints.clear()
for sc in cd.sequenceConstraints:
    print(sc.subject)
for sa in cd.sequenceAnnotations:
    if sa.component is not None:
        print(type(sa.component))
for sa in cd.sequenceAnnotations:
    if sa.identity == "http://www.dummy.org/design/design_SequenceAnnotation/1":
        cd.sequenceAnnotations.remove(sa.identity)
    else: print(sa)
cd.components.remove("http://www.dummy.org/design/CDS_Component/1")
for c in cd.components:
    print(c)

http://www.dummy.org/help
http://www.dummy.org/help/1
help
me
me
http://www.dummy.org/design/CDS_Component/1
http://www.dummy.org/design/RBS_Component/1
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
http://www.dummy.org/design/design_SequenceAnnotation1/1
http://www.dummy.org/design/design_SequenceAnnotation2/1
http://www.dummy.org/design/RBS_Component/1
http://www.dummy.org/design/Pro_Component/1
http://www.dummy.org/design/Ter_Component/1
